In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
ratings = pd.read_csv("./user_data.csv", encoding='cp949')
ratings

,user_id,pattern_id,fave_date
0,Crocheturlay,720780,2017/02/19 15:08:49 -0500
1,cozykrisknits,393166,2014/12/25 14:31:01 -0500
2,cozykrisknits,447781,2014/12/25 14:29:06 -0500
3,cozykrisknits,117296,2013/01/07 22:42:44 -0500
4,cozykrisknits,224811,2013/01/07 22:39:07 -0500
...,...,...,...
3756709,alchemia,173758,2013/03/28 16:34:16 -0400
3756710,alchemia,195006,2013/03/28 16:34:07 -0400
3756711,alchemia,143595,2013/03/28 16:34:02 -0400
3756712,alchemia,181436,2013/03/28 16:33:52 -0400


In [3]:
# 좋아요 두개 이상 누른 사용자 
much_liker = ratings.groupby(['user_id', 'pattern_id']).count() 
bool_liker = much_liker['fave_date'] > 1
much_liker = much_liker[bool_liker]
print(much_liker)

                             fave_date
user_id          pattern_id           
A-Bear           5708                2
                 13085               2
A-Ko-Cloudartowl 165798              2
                 188720              2
                 293756              2
...                                ...
curlyredheadgirl 159043              2
                 487111              2
cushing          48126               2
cute2go          395263              2
cutevira         725964              2

[17751 rows x 1 columns]


In [4]:
# 정말 이런식으로 A-Bear처럼 두개씩 누른 사람들이 있다
user = ratings['user_id'] == 'A-Bear' 
pat = ratings['pattern_id'] == 5708

print(ratings[pat & user])

        user_id  pattern_id                  fave_date
3547964  A-Bear        5708  2009/03/05 22:02:47 -0500
3547966  A-Bear        5708  2009/03/05 22:00:56 -0500


In [5]:
# 중복 제거
ratings = ratings.drop_duplicates(['user_id', 'pattern_id'], keep='last')

In [6]:
# A-Bear가 5708을 누른 데이터도 이제 한개뿐
user = ratings['user_id'] == 'A-Bear' 
pat = ratings['pattern_id'] == 5708
print(ratings[pat & user])

        user_id  pattern_id                  fave_date
3547966  A-Bear        5708  2009/03/05 22:00:56 -0500


In [7]:
users = ratings.groupby('user_id')['user_id'].count()
print(users)

user_id
A-Bear               291
A-Jar-Of-Bees         23
A-KN                 615
A-Kelli               65
A-Ko-Cloudartowl    3686
                    ... 
cvilleknits           38
cvitt                 24
cvivianay             44
cvjunebug            207
cvkasdan              16
Name: user_id, Length: 17636, dtype: int64


In [8]:
patterns = ratings.groupby('pattern_id')['pattern_id'].count()
print(patterns)

pattern_id
10        157
13         27
16        171
17        305
20        123
         ... 
774662      1
774666      1
774667      1
774670      1
774676      1
Name: pattern_id, Length: 410633, dtype: int64


In [9]:
# 신뢰할만한 user를 걸러내자.
# 어떤 user가 평가한 패턴의 개수가 N개 이상이라면, 이 user는 여러 개의 패턴을 보고 평가한 것으로 볼 수 있다.
# 그러므로 이 사용자의 평가는 믿을만하다고 가정한다.
N = 100
reliable_users = users[users > N]
reliable_users = reliable_users.to_frame()
# index와, 첫번째 column의 이름이 user_id라 pd.merge 연산이 불가하므로 column name 치환
reliable_users.rename(columns={'user_id':'count'}, inplace=True)
print(reliable_users)

                  count
user_id                
A-Bear              291
A-KN                615
A-Ko-Cloudartowl   3686
A-L                 192
A2Knitzi            280
...                 ...
cutiepie3000        150
cutiepiemommy       107
cutikula            226
cutloose            458
cvjunebug           207

[5630 rows x 1 columns]


In [10]:
# 위에서 거른 믿을만한 사용자 집단과 rating set의 교집합을 걸러내어,
# 믿을만한 사용자 집단이 평가하지 않은 pattern id는 dataset에서 제외한다.
merge_ratings = pd.merge(ratings, reliable_users, on=['user_id'], how='inner')

In [11]:
merge_ratings

,user_id,pattern_id,fave_date,count
0,Bibicoco,193187,2017/04/29 07:32:06 -0400,111
1,Bibicoco,277368,2017/04/02 00:48:07 -0400,111
2,Bibicoco,241913,2016/04/23 23:55:12 -0400,111
3,Bibicoco,368955,2015/10/12 21:39:27 -0400,111
4,Bibicoco,547501,2015/10/12 21:35:50 -0400,111
...,...,...,...,...
3465470,asmaloy,186328,2011/01/07 07:09:58 -0500,128
3465471,asmaloy,40292,2011/01/07 06:43:35 -0500,128
3465472,asmaloy,146846,2011/01/05 07:29:34 -0500,128
3465473,asmaloy,215288,2011/01/01 13:38:16 -0500,128


In [12]:
# 교집합 연산이 잘 되었는지 확인을 위한 작업
# 교집합 연산 전 reliable users 의 row 길이와, 
# 현재 merge된 ratings에서 user id끼리 groupby한 연산의 결과가 같으므로, 
# 이는 옳게 교집합 연산이 되었다
users = merge_ratings.groupby('user_id')['user_id'].count()
print(users)

user_id
A-Bear               291
A-KN                 615
A-Ko-Cloudartowl    3686
A-L                  192
A2Knitzi             280
                    ... 
cutiepie3000         150
cutiepiemommy        107
cutikula             226
cutloose             458
cvjunebug            207
Name: user_id, Length: 5630, dtype: int64


In [13]:
patterns = merge_ratings.groupby('pattern_id')['pattern_id'].count()
print(patterns)
# 그러나, 사람마다 취향이 너무 달라서 제외된 패턴임에도 row가 410000개이다. 여전히 너무 많아 MF를 실행할 수 없다.

pattern_id
10        143
13         25
16        148
17        283
20        109
         ... 
774662      1
774666      1
774667      1
774670      1
774676      1
Name: pattern_id, Length: 402052, dtype: int64


In [14]:
# 이젠 신뢰할만한 pattern을 걸러내자.
# 어떤 pattern이 평가된 횟수가 M개 이상이라면, 이 패턴은 많은 사용자에게 평가받았다.
# 그러므로 이 패턴은 보편적 취향에 부합하며, 다른 이에게도 추천할만하다.
M = 100
reliable_patterns = patterns[patterns > M]
reliable_patterns = reliable_patterns.to_frame()
reliable_patterns.rename(columns={'pattern_id':'count'}, inplace=True)
print(reliable_patterns)

            count
pattern_id       
10            143
16            148
17            283
20            109
29            584
...           ...
761594        116
763023        115
763263        112
763264        130
766149        106

[3859 rows x 1 columns]


In [15]:
# 위에서 거른 믿을만한 사용자 집단과 rating set의 교집합을 걸러내어,
# 믿을만한 사용자 집단이 평가하지 않은 pattern id는 dataset에서 제외한다.
s_merge_ratings = pd.merge(merge_ratings, reliable_patterns, on=['pattern_id'], how='inner')
# s_merge_ratings = s_merge_ratings.drop(columns = ['fave_date','count_x', 'count_y'])
s_merge_ratings = s_merge_ratings.drop(columns = ['count_x', 'count_y'])

In [16]:
# user는 줄지 않았다
users = s_merge_ratings.groupby('user_id')['user_id'].count()
print(users)

user_id
A-Bear               67
A-KN                 98
A-Ko-Cloudartowl    450
A-L                  75
A2Knitzi             86
                   ... 
cutiepie3000          6
cutiepiemommy         9
cutikula             41
cutloose             59
cvjunebug            10
Name: user_id, Length: 5622, dtype: int64


In [17]:
# 교집합 연산이 잘 되었는지 확인을 위한 작업
# 교집합 연산 전 reliable patterns 의 row 길이와, 
# 현재 merge된 ratings에서 pattern id끼리 groupby한 연산의 결과가 같으므로, 
# 이는 옳게 교집합 연산이 되었다
patterns = s_merge_ratings.groupby('pattern_id')['pattern_id'].count()
print(patterns)

pattern_id
10        143
16        148
17        283
20        109
29        584
         ... 
761594    116
763023    115
763263    112
763264    130
766149    106
Name: pattern_id, Length: 3859, dtype: int64


In [18]:
print(s_merge_ratings)

                 user_id  pattern_id                  fave_date
0               Bibicoco      277368  2017/04/02 00:48:07 -0400
1         churncreeklady      277368  2014/05/15 13:10:56 -0400
2           anneliesbaes      277368  2014/08/15 04:08:53 -0400
3       ConstanceTricote      277368  2012/10/28 11:30:45 -0400
4                 Ann357      277368  2015/07/22 16:00:25 -0400
...                  ...         ...                        ...
732657         Brewst502      164869  2010/11/25 18:06:12 -0500
732658   charliehrtsmatt      164869  2012/05/01 23:47:49 -0400
732659        ArgyleLove      164869  2016/07/17 17:21:11 -0400
732660        badpallone      164869  2010/10/11 09:18:01 -0400
732661            aerynn      164869  2011/01/22 22:39:15 -0500

[732662 rows x 3 columns]


In [19]:
# print(min(s_merge_ratings['fave_date']), max(s_merge_ratings['fave_date']))

# s_merge_ratings['fave_date'] = pd.to_datetime(s_merge_ratings['fave_date'])
print(min(s_merge_ratings['fave_date']), max(s_merge_ratings['fave_date']))

s_merge_ratings.loc[s_merge_ratings['fave_date'] <= '2017/12/31', 'score'] = 5
s_merge_ratings.loc[s_merge_ratings['fave_date'] <= '2015/12/31', 'score'] = 4
s_merge_ratings.loc[s_merge_ratings['fave_date'] <= '2013/12/31', 'score'] = 3
s_merge_ratings.loc[s_merge_ratings['fave_date'] <= '2011/12/31', 'score'] = 2
s_merge_ratings.loc[s_merge_ratings['fave_date'] <= '2009/12/31', 'score'] = 1

2007/09/27 10:10:22 -0400 2017/09/16 05:22:01 -0400


In [20]:
s_merge_ratings = s_merge_ratings.drop(columns = ['fave_date'])

In [21]:
# s_merge_ratings['values'] = 1

s_merge_ratings = s_merge_ratings.pivot( index='user_id', columns='pattern_id')
forcopy = pd.DataFrame(s_merge_ratings)
print(s_merge_ratings)


                  score                                                   \
pattern_id       10     16     17     20     29     38     40     45       
user_id                                                                    
A-Bear              NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
A-KN                NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
A-Ko-Cloudartowl    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
A-L                 NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
A2Knitzi            NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
...                 ...    ...    ...    ...    ...    ...    ...    ...   
cutiepie3000        NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
cutiepiemommy       NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
cutikula            NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
cutloose            NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
cvjunebug   

In [22]:
s_merge_ratings = s_merge_ratings.fillna(0)
s_merge_ratings = s_merge_ratings.to_numpy()
s_merge_ratings.to_csv("./sgd.csv")
print(s_merge_ratings)

AttributeError: 'numpy.ndarray' object has no attribute 'to_csv'

In [ ]:
from sklearn.metrics import mean_squared_error
 
def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적 곱으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
     
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
     
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
       
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
     
    return rmse
 
 
def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda = 0.01):
    num_users, num_items = R.shape
    # P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 랜덤한 값으로 입력합니다.
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))
 
    break_count = 0
    # R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장.
    non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]
     
    # SGD기법으로 P와 Q 매트릭스를 계속 업데이트.
    for step in range(steps):
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이인 오류 값 구함
            eij = r - np.dot(P[i, :], Q[j, :].T)
            # Regularization을 반영한 SGD 업데이트 공식 적용
            P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])
         
        rmse = get_rmse(R, P, Q, non_zeros)
        if step % 10 == 0:
            print("### iteration step : ", step," rmse : ", np.round(rmse, 7))
             
    return P, Q

In [ ]:
'''#R = pd.DataFrame(s_merge_ratings)
s_merge_ratings = pd.DataFrame(s_merge_ratings)
#ratings_matrix = R.pivot_table()

P, Q = matrix_factorization(s_merge_ratings.values, K=30, steps=41, learning_rate=0.01, r_lambda = 0.01)'''

In [ ]:
'''pred_matrix = np.dot(P, Q.T) # P @ Q.T 도 가능
print('실제 행렬:\n', s_merge_ratings)
print('\n예측 행렬:\n', np.round(pred_matrix, 2))'''

In [ ]:
# another type SGD code

import numpy as np
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization():
    def __init__(self, R, k, learning_rate, reg_param, epochs, verbose=False):
        """
        :param R: rating matrix
        :param k: latent parameter
        :param learning_rate: alpha on weight update
        :param reg_param: beta on weight update
        :param epochs: training epochs
        :param verbose: print status
        """
        self._R = R
        self._num_users, self._num_items = R.shape
        self._k = k
        self._learning_rate = learning_rate
        self._reg_param = reg_param
        self._epochs = epochs
        self._verbose = verbose


    def fit(self):
        """
        training Matrix Factorization : Update matrix latent weight and bias

        참고: self._b에 대한 설명
        - global bias: input R에서 평가가 매겨진 rating의 평균값을 global bias로 사용
        - 정규화 기능. 최종 rating에 음수가 들어가는 것 대신 latent feature에 음수가 포함되도록 해줌.

        :return: training_process
        """

        # init latent features
        self._P = np.random.normal(size=(self._num_users, self._k))
        self._Q = np.random.normal(size=(self._num_items, self._k))

        # init biases
        self._b_P = np.zeros(self._num_users)
        self._b_Q = np.zeros(self._num_items)
        self._b = np.mean(self._R[np.where(self._R != 0)])

        # train while epochs
        self._training_process = []
        for epoch in range(self._epochs):
            # rating이 존재하는 index를 기준으로 training
            xi, yi = self._R.nonzero()
            for i, j in zip(xi, yi):
                self.gradient_descent(i, j, self._R[i, j])
            cost = self.cost()
            self._training_process.append((epoch, cost))

            # print status
            if self._verbose == True and ((epoch + 1) % 5 == 0):
                print("Iteration: %d ; cost = %.4f" % (epoch + 1, cost))


    def cost(self):
        """
        compute root mean square error
        :return: rmse cost
        """

        # xi, yi: R[xi, yi]는 nonzero인 value를 의미한다.
        # 참고: http://codepractice.tistory.com/90
        xi, yi = self._R.nonzero()
        # predicted = self.get_complete_matrix()
        cost = 0
        for x, y in zip(xi, yi):
            cost += pow(self._R[x, y] - self.get_prediction(x, y), 2)
        return np.sqrt(cost/len(xi))


    def gradient(self, error, i, j):
        """
        gradient of latent feature for GD

        :param error: rating - prediction error
        :param i: user index
        :param j: item index
        :return: gradient of latent feature tuple
        """

        dp = (error * self._Q[j, :]) - (self._reg_param * self._P[i, :])
        dq = (error * self._P[i, :]) - (self._reg_param * self._Q[j, :])
        return dp, dq


    def gradient_descent(self, i, j, rating):
        """
        graident descent function

        :param i: user index of matrix
        :param j: item index of matrix
        :param rating: rating of (i,j)
        """

        # get error
        prediction = self.get_prediction(i, j)
        error = rating - prediction

        # update biases
        self._b_P[i] += self._learning_rate * (error - self._reg_param * self._b_P[i])
        self._b_Q[j] += self._learning_rate * (error - self._reg_param * self._b_Q[j])

        # update latent feature
        dp, dq = self.gradient(error, i, j)
        self._P[i, :] += self._learning_rate * dp
        self._Q[j, :] += self._learning_rate * dq


    def get_prediction(self, i, j):
        """
        get predicted rating: user_i, item_j
        :return: prediction of r_ij
        """
        return self._b + self._b_P[i] + self._b_Q[j] + self._P[i, :].dot(self._Q[j, :].T)


    def get_complete_matrix(self):
        """
        computer complete matrix PXQ + P.bias + Q.bias + global bias

        - PXQ 행렬에 b_P[:, np.newaxis]를 더하는 것은 각 열마다 bias를 더해주는 것
        - b_Q[np.newaxis:, ]를 더하는 것은 각 행마다 bias를 더해주는 것
        - b를 더하는 것은 각 element마다 bias를 더해주는 것

        - newaxis: 차원을 추가해줌. 1차원인 Latent들로 2차원의 R에 행/열 단위 연산을 해주기위해 차원을 추가하는 것.

        :return: complete matrix R^
        """
        return self._b + self._b_P[:, np.newaxis] + self._b_Q[np.newaxis:, ] + self._P.dot(self._Q.T)

In [ ]:
factorizer = MatrixFactorization(s_merge_ratings, k=30, learning_rate=0.05, reg_param=0.02, epochs=1, verbose=True)
factorizer.fit()

In [ ]:
mtrx = factorizer.get_complete_matrix()

In [ ]:
mtrx = pd.DataFrame(mtrx)
mtrx = np.round(mtrx, 5)

In [ ]:
print(patterns.index, users.index)

In [ ]:
# mtrx의 column은 patternId, row는 userId로 바꾸기 
mtrx.columns = patterns.index
mtrx.index = users.index
mtrx.columns.name = "patternId"
mtrx.index.name = "userId"
mtrx.tail(5)

In [ ]:
new_user = "akinakamori"
new_pattern = 10
data = {
    new_pattern: '5' # '5' means score
}
df = pd.DataFrame(data, index=[new_user])
appended_mtrx = mtrx.append(df)

# 데이터 추가해서 원래 데이터프레임에 저장하기
#appended_mtrx = mtrx.append(data_to_insert)
appended_mtrx.tail(5)

In [ ]:
# mtrx.to_csv("./sgd.csv")

In [ ]:
import pickle, joblib
 
## Save pickle
with open("cf.pickle","wb") as fw:
    pickle.dump(factorizer, fw)

In [ ]:
saved_model = pickle.dumps(factorizer)
ft_from_pickle = pickle.loads(saved_model)
ft_from_pickle.get_complete_matrix()

In [ ]:
joblib.dump(factorizer, 'cf_factorization.pkl') 
ft_from_joblib = joblib.load('cf_factorization.pkl') 
ft_from_joblib.get_complete_matrix()